# setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
import geopandas as gpd
import xarray as xr
from pymannkendall import original_test

shp_pan = gpd.read_file(r'Data\shapefiles\panamz.geojson')


In [2]:
folder_ts = r'Data\Datasets\amz\ts'
folder_metric = r'Data\Datasets\amz\seasonal'
datasets = ['cru', 'gpcc', 'chirps','imerg', 'terra', 'era_land', 'jra55','merra2']
datasets_names = ['CRU', 'GPCC', 'CHIRPS','IMERG-V6', 'TerraClimate', 'ERA5-Land', 'JRA55','MERRA2']


In [12]:
# functions

def seasons_ts(ds):
    seasons = {
        'DJF': [12, 1, 2],
        'MAM': [3, 4, 5],
        'JJA': [6, 7, 8],
        'SON': [9, 10, 11]
    }
    # Resample the dataset to seasonal frequency
    seasonal_dataset = ds.resample(time='Q-NOV').mean()
    # Create empty lists to store the seasonal datasets
    seasonal_datasets = []
    # Iterate over each season and extract the precipitation data
    for season, months in seasons.items():
        # Select the months corresponding to the current season
        seasonal_data = seasonal_dataset.sel(time=seasonal_dataset['time.month'].isin(months))

        # Group the data by year and sum the precipitation
        seasonal_data_grouped = seasonal_data.groupby('time.year').mean(dim='time')

        # Append the seasonal dataset to the list
        seasonal_datasets.append(seasonal_data_grouped)

    # Concatenate the seasonal datasets along a new 'season' dimension
    seasonal_precipitation = xr.concat(seasonal_datasets, dim='season')

    # Add the 'season' coordinate
    seasonal_precipitation['season'] = list(seasons.keys())
    seasonal_precipitation = seasonal_precipitation.sel(year=slice(2002,2019))
    return seasonal_precipitation

# functions
def mannkendall_trend(arr):
    if not np.isnan(arr).any():
        result = original_test(arr)
        return result.p, result.slope, result.intercept
    else:
        return np.nan, np.nan, np.nan

def ds_kendall(data, dim):
    results =  xr.apply_ufunc(mannkendall_trend, data,
                             input_core_dims=[[dim]],
                             output_core_dims=[[], [],[]],
                             vectorize=True,
                             dask='parallelized')
    
    
    # Extract the p-values and Sen's slopes from the results
    p_values = results[0]
    slopes = results[1]
    intercepts = results[2]
    
    # Create a new xarray dataset to store the results
    results_dataset = xr.Dataset({'p_values': p_values.pr, 'slopes': slopes.pr, 'intercepts': intercepts.pr})
    return results_dataset



In [7]:
for dataset in datasets:
    #read file of dataset in folder_clean
    file_path = glob(os.path.join(folder_ts, dataset + '.nc'))
    ds = xr.open_dataset(file_path[0])
    ds_ts = seasons_ts(ds)
    ds_clim = ds_ts.mean('year')
    ds_trend = ds_kendall(ds_ts, 'year')
    #save
    ds_ts.to_netcdf(os.path.join(folder_metric, dataset + '.nc'))
    ds_clim.to_netcdf(os.path.join(folder_metric,'clim', dataset + '.nc'))
    ds_trend.to_netcdf(os.path.join(folder_metric,'trend', dataset + '.nc'))

# station

In [4]:
stations = gpd.read_file(r'Data\Evaluation\stations_amz_ANA.geojson')
df_stat= pd.read_pickle(r'Data\Evaluation\amz_01_20_20bet.pkl')

In [12]:
stations

,Code,Name,Latitude,Longitude,Date_begin,Date_end,index_right,Region,geometry,trend_DJF,...,trend_JJA,p_JJA,slope_JJA,intercept_JJA,clim_JJA,trend_SON,p_SON,slope_SON,intercept_SON,clim_SON
0,1.500000e+01,Station 15,-76.779440,-6.915000,1981-01-01,2019-02-01,3,Western,POINT (-76.77944 -6.91500),no trend,...,no trend,0.324712,-1.383333,77.823908,69.602689,no trend,0.544486,-0.976153,145.097300,137.710975
1,2.900000e+01,Station 29,-76.321940,-6.593056,1981-01-01,2019-02-01,3,Western,POINT (-76.32194 -6.59306),no trend,...,no trend,0.595912,-0.275000,59.270833,57.632005,no trend,0.255818,-0.674286,101.298122,96.632670
2,5.000000e+01,Station 50,-77.166940,-6.046667,1986-06-01,2019-02-01,3,Western,POINT (-77.16694 -6.04667),no trend,...,no trend,0.404670,0.810648,58.223967,66.195709,no trend,1.000000,-0.062206,147.647747,145.679969
3,6.500000e+01,Station 65,-75.846940,-9.152500,1998-09-01,2019-02-01,3,Western,POINT (-75.84694 -9.15250),no trend,...,no trend,0.969764,-0.006667,81.073333,82.463939,no trend,0.879573,0.610589,210.609990,226.328444
4,6.600000e+01,Station 66,-76.084720,-9.677778,1994-09-01,2018-08-01,3,Western,POINT (-76.08472 -9.67778),no trend,...,no trend,0.495366,-0.645049,94.870394,87.444036,no trend,1.000000,-0.082416,185.475598,187.120324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,1.269000e+09,La Asunta,-67.196667,-16.126667,2002-01-01,2020-11-01,2,Southern,POINT (-67.19667 -16.12667),no trend,...,no trend,0.704853,0.572423,34.052986,38.866568,no trend,0.595912,-1.083333,93.614630,90.626176
577,1.270000e+09,Incapampa,-67.728056,-16.225556,2004-01-01,2020-11-01,2,Southern,POINT (-67.72806 -16.22556),no trend,...,no trend,0.495366,0.685383,46.048372,55.440909,no trend,0.622180,-0.973409,111.773978,110.590508
578,1.273000e+09,Sorata,-68.651667,-15.766667,1999-12-01,2020-11-01,2,Southern,POINT (-68.65167 -15.76667),no trend,...,no trend,0.129745,0.728975,14.552440,23.448120,no trend,0.448718,-0.821383,71.148422,62.801214
579,1.392000e+09,San Juan de YapacanÃ­,-63.833333,-17.250000,1999-12-01,2020-11-01,2,Southern,POINT (-63.83333 -17.25000),no trend,...,no trend,0.324712,0.710582,58.341750,69.275670,no trend,0.160772,3.000000,101.935168,130.287898


In [5]:
df_stat_sea = df_stat.groupby('Code')[['Total', 'Date']].resample('Q-NOV', on='Date').mean().reset_index()
df_stat_sea['season'] = df_stat_sea.Date.dt.month.apply(lambda x: 'DJF' if x in [12, 1, 2] else ('MAM' if x in [3, 4, 5] else ('JJA' if x in [6, 7, 8] else 'SON')))
#just from 2002 to 2019 and drop Date Column
df_stat_sea = df_stat_sea[df_stat_sea.Date.dt.year.isin(range(2002,2020))]
#create year column
df_stat_sea['year'] = df_stat_sea.Date.dt.year
df_stat_sea = df_stat_sea.drop('Date', axis=1)
df_stat_sea_clim = df_stat_sea.groupby(['Code', 'season']).mean().reset_index()

In [8]:
# do a mk test for each season and each code
for season in df_stat_sea.season.unique():
    for code in df_stat_sea.Code.unique():
        test = original_test(df_stat_sea[(df_stat_sea.season == season) & (df_stat_sea.Code == code)]['Total'])
        #create a column for each season
        stations.loc[stations.Code == code, 'trend'+ '_' + season] = test.trend
        stations.loc[stations.Code == code, 'p'+ '_' + season] = test.p
        stations.loc[stations.Code == code, 'slope'+'_' +  season] = test.slope
        stations.loc[stations.Code == code, 'intercept'+ '_' + season] = test.intercept
        #create a column for each clim
        stations.loc[stations.Code == code, 'clim'+ '_' + season] = df_stat_sea_clim[(df_stat_sea_clim.season == season) & (df_stat_sea_clim.Code == code)]['Total'].values[0]

In [11]:
stations.to_file(os.path.join(folder_metric, 'stations.geojson'), driver='GeoJSON')
#df_ts.to_csv(os.path.join(folder_metric, 'stations_ts.csv'))